# 3 start auth pods on minikube

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/environments/minikube/env

## start authorization & authentication service on minikube

### create `secrets/auth-tokens.json`

example)
```json
[
    {
        "host": ".*",
        "settings": {
            "bearer_tokens": [
                {
                    "token": "1IqNHfjQsD84mPHvciATObXM3ozfHmX1",
                    "allowed_paths": ["^/orion/.*$", "^/idas/.*$", "^/comet/.*$"]
                }
            ],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": []
            }
        }
    }
]
```

#### for macOS

In [ ]:
cat << __EOS__ > secrets/auth-tokens.json
[
    {
        "host": ".*",
        "settings": {
            "bearer_tokens": [
                {
                    "token": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)",
                    "allowed_paths": ["^/orion/.*$", "^/idas/.*$", "^/comet/.*$"]
                }
            ],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": []
            }
        }
    }
]
__EOS__

#### for Ubuntu

In [ ]:
cat << __EOS__ > secrets/auth-tokens.json
[
    {
        "host": ".*",
        "settings": {
            "bearer_tokens": [
                {
                    "token": "$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32)",
                    "allowed_paths": ["^/orion/.*$", "^/idas/.*$", "^/comet/.*$"]
                }
            ],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": []
            }
        }
    }
]
__EOS__

### register auth-tokens to kubernetes secrets

In [ ]:
kubectl create secret generic auth-tokens --from-file=./secrets/auth-tokens.json

### start auth service

In [ ]:
kubectl apply -f auth/auth-service.yaml

In [ ]:
kubectl apply -f auth/auth-deployment.yaml

In [ ]:
kubectl get pods -l app=auth

example)
```
NAME                    READY   STATUS    RESTARTS   AGE
auth-67ff6bd94b-7sxt2   1/1     Running   0          15s
auth-67ff6bd94b-98svn   1/1     Running   0          15s
auth-67ff6bd94b-xrfpz   1/1     Running   0          15s
```

In [ ]:
kubectl get services -l app=auth

example)
```
NAME   TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
auth   ClusterIP   10.99.182.149   <none>        3000/TCP   41s
```